In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

# Ignore all GPUs, tf random forest does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
# Parameters
num_steps = 50 # Total steps to train
batch_size = 128 # The number of samples per batch
num_classes = 3 # NSP
num_features = 21
num_trees = 10
max_nodes = 100
df = pd.read_csv('input/test.csv')
df.head(5)
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [3]:
inputDF = df[df.columns[:num_features]]
inputDF = inputDF.astype(np.float32)
labelDF = df[df.columns[-1]]
labelDF = labelDF.astype(np.int32)

In [4]:
inputDF.tail(5)


,LB,AC,FM,UC,DL,DP,ASTV,MSTV,ALTV,MLTV,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,CLASS
2121,140.0,0.000,0.000,0.007,0.0,0.0,79.0,0.2,25.0,7.2,...,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0,5.0
2122,140.0,0.001,0.000,0.007,0.0,0.0,78.0,0.4,22.0,7.1,...,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0,5.0
2123,140.0,0.001,0.000,0.007,0.0,0.0,79.0,0.4,20.0,6.1,...,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0,5.0
2124,140.0,0.001,0.000,0.006,0.0,0.0,78.0,0.4,27.0,7.0,...,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0,5.0
2125,142.0,0.002,0.002,0.008,0.0,0.0,74.0,0.4,36.0,5.0,...,117.0,159.0,2.0,1.0,145.0,143.0,145.0,1.0,0.0,1.0


In [5]:
labelDF.shape

(2126,)

In [6]:
# Input and Target data
data = tf.placeholder(tf.float32, shape=[None, num_features])
# For random forest, labels must be integers (the class id)
target = tf.placeholder(tf.int32, shape=[None])

In [7]:
# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

classifier = tensor_forest.RandomForestGraphs(hparams)







INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 10, 'max_nodes': 100, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 10, 'max_fertile_nodes': 0, 'split_after_samples': 250, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'model_name': 'all_dense', 'split_finish_name': 'basic', 'split_pruning_name': 'none', 'collate_examples': False, 'checkpoint_stats': False, 'use_running_stats_method': False, 'initialize_average_splits': False, 'inference_tree_paths': False, 'param_file': None, 'split_name': 'less_or_equal', 'early_finish_check_every_samples': 0, 'prune_every_samples': 0, 'num_classes': 3, 'num_features': 21, 'bagged_num_features': 21, 'bagged_features': None, 'regression': False, 'num_outputs': 1, 'num_output_columns': 4, 'base_random_seed': 0, 'leaf_model_type': 0, 'stats_model_type': 0, 'finish_type': 0, 'pruning_type': 0, 'split_type': 0}


In [8]:
#Get training graph and loss
train_op = classifier.training_graph(inputDF,labelDF)
loss_op = classifier.training_loss(inputDF,labelDF)

In [9]:
#Measure the accuracy
infer_op, _, _ = classifier.inference_graph(inputDF)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(labelDF, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
#Initilize variables
init_vars = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_vars)

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)
type(mnist)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [11]:
# Training
for i in range(1, num_steps + 1):
    # Predate data
    batch_x = inputDF.train.next_batch(batch_size)
    batch_y = labelDF.train.next_batch(batch_size)

AttributeError: 'DataFrame' object has no attribute 'train'